In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from tqdm import tqdm

### 데이터 압축해제

In [ ]:
# 라벨링데이터 압축 풀기
%cd /content/drive/MyDrive
!unzip -qq "/content/drive/MyDrive/determining_code_similarity_AI_competition-20220612T174726Z-001.zip"

In [ ]:
# # 라벨링데이터 압축 풀기
# %cd /content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/label
# !unzip -qq "/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/label_data.zip"

In [ ]:
# # 원천데이터 압축 풀기
# %cd /content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/image
# !unzip -qq "/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/image_data.zip"

In [ ]:
# 데이터 개수 확인
import os

def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)
	
dir_path_label = "/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/label"
dir_path_image = "/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/image"
cnt_label = 0
cnt_image = 0
fcnt_label = {}
fcnt_image = {}

for (root, directories, files) in tqdm(os.walk(dir_path_label)):
  for d in directories:
    f_l = get_files_count(os.path.join(root, d))
    fcnt_label[d] = f_l
    cnt_label += f_l

for (root, directories, files) in tqdm(os.walk(dir_path_image)):
  for d in directories:
    f_i = get_files_count(os.path.join(root, d))
    fcnt_image[d] = f_i
    cnt_image += f_i

print(fcnt_label)
print(fcnt_image)

for key in fcnt_label.keys():
  if fcnt_label[key] != fcnt_image[key]:
    print(fcnt_label[key],",",fcnt_image[key])
print("전체 레이블 수: ",cnt_label,"\t전체 이미지 수: ",cnt_image)

In [ ]:
# # 오류났을 경우 폴더 삭제
# import os
# import shutil

# dir_path = "/content/drive/MyDrive/deep-learning-from-scratch-master"

# if os.path.exists(dir_path):
#     shutil.rmtree(dir_path)
# os.mkdir("/content/drive/MyDrive/deep-learning-from-scratch-master")

### 데이터 전처리

In [ ]:
import os
import numpy as np
import pandas as pd
import json #json을 파싱하기 위해
import re
from sklearn.model_selection import train_test_split

In [ ]:
image_df = pd.read_csv('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/image_df.csv', encoding='utf-8-sig')

In [ ]:
dataset_df = pd.read_csv('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/dataset_df.csv', encoding='utf-8-sig')

In [ ]:
dataset_df.drop(["Unnamed: 0","라벨링_아우터","라벨링_하의","라벨링_원피스","라벨링_상의"], axis=1, inplace=True)

In [ ]:
dataset_df.head()

In [ ]:
# '스타일'열 정리
df = dataset_df
labels = []
for r in df.index:
  spdf = df.loc[r,'스타일'].split(':')
  df.at[r,'스타일'] = re.findall('[가-힣]+', spdf[1].split(',')[0])[0]
  labels.append(df.loc[r,'스타일'])

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# 레이블 인코딩
encoder = LabelEncoder()
encoder.fit(labels)
encoded_labels = encoder.transform(labels)
encoded_labels = encoded_labels.reshape(-1,1)
# ont-hot 인코딩
oh_encoder = OneHotEncoder()
oh_encoder.fit(encoded_labels)
oh_labels = oh_encoder.transform(encoded_labels)
oh_labels = oh_labels.toarray()
print(oh_labels.shape)

In [ ]:
# label data
Y = oh_labels

Y_train, Y_test = train_test_split(Y,
                                  random_state=100,
                                  test_size=0.1,
                                  stratify=Y,)
# 파일로 저장
np.save('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/style_classification/Y_train', Y_train)
np.save('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/style_classification/Y_test', Y_test)

###모델 쌓기

In [ ]:
X_train = np.load('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/category_classification/X_train.npy')
X_test = np.load('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/category_classification/X_test.npy')
Y_train = np.load('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/style_classification/Y_train.npy')
Y_test = np.load('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/style_classification/Y_test.npy')

In [ ]:
import tensorflow as tf
import keras
from keras import models, layers, Input, optimizers, initializers, regularizers, metrics, losses
from keras.models import Model, Sequential
from keras.applications.resnet_v2 import ResNet50V2, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
model = ResNet50V2(weights=None, input_tensor=Input(shape=(100,100,3)), classes=9)

In [ ]:
input_shape = (100, 100, 3)
drop_rate = 0.5

In [ ]:
model.compile(optimizer='adam',
              loss=losses.categorical_crossentropy,
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=128,
                    epochs=20,
                    validation_data=(X_test, Y_test))
model.save("/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/style_classification/style_model1")

In [ ]:
model = keras.models.load_model("/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/style_classification/style_model1")

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history["val_accuracy"]

plt.plot(range(1, len(acc)+1), acc, label="train")
plt.plot(range(1, len(acc)+1), val_acc, label="test")
plt.legend()

In [ ]:
img_path = '/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/조끼.jpg'
img = image.load_img(img_path, target_size=(100, 100), interpolation='nearest')
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# print('Predicted:', decode_predictions(preds, top=3)[0])
print(np.argmax(preds))